In [1]:
!pip3 install numpy nltk

     |████████████████████████████████| 1.5MB 3.4MB/s eta 0:00:01
  Stored in directory: /Users/donatas/Library/Caches/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [5]:
# python3.5 buildtagger.py <train_file_absolute_path> <model_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def train_model(train_file, model_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    train_file = sys.argv[1]
    model_file = sys.argv[2]
    start_time = datetime.datetime.now()
    train_model(train_file, model_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


Finished...
Time: 0:00:00.000121


In [5]:
# python3.5 runtagger.py <test_file_absolute_path> <model_file_absolute_path> <output_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def tag_sentence(test_file, model_file, out_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    test_file = sys.argv[1]
    model_file = sys.argv[2]
    out_file = sys.argv[3]
    start_time = datetime.datetime.now()
    tag_sentence(test_file, model_file, out_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


IndexError: list index out of range

## Creating the model statistics

Need to handle special cases like these:
 - fractions like: 1/2/CD
 - multiple listings: pianist/bassoonist/composer/NN
 
 As far as I understand all of the words belong to the same POS. 
 A function that adds both words should be implemented, but an initial version function should simply take the last two elements and uses them as word + tag.
 
 
 Also, likely the score could be improved by adding add-one-smoothing

In [52]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))

class PosModelStats():
    def __init__(self, emissions, transitions, words, tags):
        self.emissions = emissions
        self.transitions = transitions
        self.words = words
        self.tags = tags

def extract_frequencies(filename):
    emission_dict = {}
    transition_dict = {}
    unique_words = set()
    unique_tags = set()

    with open(filename) as training_file:
        word_tag_pairs = [sentence.split(" ") for sentence in training_file.read().split("\n")]
        word_tag_pairs = flatten(word_tag_pairs)
        word_count = len(word_tag_pairs)
        previous_tag = word_tag_pairs[0].split("/")[-1]

        for word_tag in word_tag_pairs:
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]

            unique_words.add(word)
            unique_tags.add(tag)

            if word_tag not in emission_dict.keys():
                emission_dict[word_tag] = 1
            else:
                emission_dict[word_tag] += 1

            if previous_tag + " " + tag not in transition_dict.keys():
                transition_dict[previous_tag + " " + tag] = 1
            else:
                transition_dict[previous_tag + " " + tag] += 1

            previous_tag = tag
    
    return PosModelStats(emission_dict, transition_dict, unique_words, unique_tags)
            
def add_one_smoothing():
    pass

In [53]:
%%time

pos_model_stats = extract_frequencies('sents.train')


CPU times: user 2.5 s, sys: 47.4 ms, total: 2.55 s
Wall time: 2.6 s


In [71]:
import _pickle as pickle

def write_output(model_stats, filename="model-file"):
    with open(filename, "wb") as model_file:
        model_file.write(pickle.dumps(model_stats))

In [72]:
%%time

write_output(pos_model_stats)

CPU times: user 50.7 ms, sys: 8.89 ms, total: 59.6 ms
Wall time: 62.8 ms


In [65]:
with open("model-file", "rb") as model_read:
    loaded = pickle.load(model_read)